# resultsVol_1: Getting data

Anish ran synapse detection on the chessboard dataset and has provided locations of potential synapses.
Here, we pull data from those locations and export it into a nice dataset for further analysis.

In [8]:
import csv
import ndio
import numpy as np
import h5py
import scipy.linalg
from numpy import genfromtxt
import ndio.remote.neurodata as neurodata
import os
import sys

nd = neurodata(hostname = "synaptomes.neurodata.io")
local = False

TOKEN1 = "kristina15"##(sys.argv[1])##"Ex10R55"
#INPUT = "./LocationData/Ex12R75/resultVol_1.csv"##(sys.argv[2])
INPUT = "K15F0_rscaled317samp5e3_hmc5NN.csv" # Locations from meda hmc
NAME = "synaptograms"##(sys.argv[3])

bf = 5

output = TOKEN1 + "rscaled317samp5000_" + NAME + "11cubes5NN_test3"+ ".h5"
output

'kristina15rscaled317samp5000_synaptograms11cubes5NN_test3.h5'

### User defined functions

In [9]:
execfile("/Users/JLP/neurodata/synaptome-stats/Code/Python/functions.py")

In [10]:
public_tokens = nd.get_public_tokens()

In [11]:
TOKEN1 in public_tokens # Should *definitely* be true

False

In [38]:
size = nd.get_image_size(TOKEN1)
channels = nd.get_channels(TOKEN1)

xGlobal = [0,size[0]]
yGlobal = [0,size[1]]
zGlobal = [0,size[2]]

globalMaxs = [size[0], size[1], size[2]]
globalMaxs

[1826, 12986, 41]

In [39]:
chan = []
for key in channels:
    chan.append(key)
del chan[3]
del chan[-5]
chan

[u'CR1_2ndA',
 u'GluR2_2ndA',
 u'PV_1stA',
 u'VAChT_4thA',
 u'GABARa1_4thA',
 u'VGluT2_2ndA',
 u'tubulin_8thA',
 u'VGluT3_1stA',
 u'TH_5thA',
 u'VGluT1_3rdA',
 u'NR2B_9thA',
 u'VGluT1_8thA',
 u'synapsinR_7thA',
 u'GAD_6thA',
 u'VGAT_5thA',
 u'Synpod_3rdA',
 u'synapsinGP_5thA',
 u'psd_8thA',
 u'dapi_1stA',
 u'GABABR1_3rdA',
 u'NMDAR1_6thA',
 u'NOS_9thA',
 u'gephyrin_1stA']

## Import resultsVol_1
Import the location data from "resultVol_1.csv" making sure that each location is not too 
close to the boundary of the data and round to nearest integer.
And that the points are far enough away from the boundary.

In [53]:
loc = genfromtxt(INPUT, delimiter=',', skip_header = 1).tolist()

L1 = []
for i in range(len(loc)):
    if inRange(loc[i], globalMaxs, bf):
        L1.append(loc[i])
        
np.random.seed(317)
LL = np.around(np.asarray(L1), decimals = 0).astype(int)
#s1 = np.random.choice(LL.shape[0], 10, replace = False)
L = LL
len(L)

40

In [54]:
xLocal = [0, max(L[:,0])]
yLocal = [0, max(L[:,1])] 
zLocal = [0, max(L[:,2])] 

## Extract 11x11x11 cube around each location 
### 11x11x11 cube is used for features F0-F3, with the buffer needed for features F4-F5
then cast to vector of length 11^3 and output as an array dim = (n,1331)

#### N.B. This does take a while

In [55]:
synCube = []
for i in range(len(L)):
    print i, "out of ", len(L)
    cubeCh = []
    for ch in chan:
        xLocal = L[i][0]
        yLocal = L[i][1]
        zLocal = L[i][2]
        queryGlobal = { 
            'token': TOKEN1,
            'channel': ch,
            'x_start': xLocal - (bf+1),
            'x_stop': xLocal + bf,
            'y_start': yLocal - (bf +1),
            'y_stop': yLocal + bf,
            'z_start': zLocal - (bf +1),
            'z_stop': zLocal + bf,
            'resolution': 0   
            }
        
        gg = nd.get_cutout(**queryGlobal)
        cubeCh.append(gg)
        out = np.asarray(cubeCh)
    synCube.append(out)

0 out of  40


KeyboardInterrupt: 

In [101]:
len(chan)

23

In [117]:
len(synCube)

3

In [57]:
#columnDat = np.transpose(np.asarray(cols))
columnNames = [str(key) for key in chan]
columnNames

['CR1_2ndA',
 'GluR2_2ndA',
 'PV_1stA',
 'VAChT_4thA',
 'GABARa1_4thA',
 'VGluT2_2ndA',
 'tubulin_8thA',
 'VGluT3_1stA',
 'TH_5thA',
 'VGluT1_3rdA',
 'NR2B_9thA',
 'VGluT1_8thA',
 'synapsinR_7thA',
 'GAD_6thA',
 'VGAT_5thA',
 'Synpod_3rdA',
 'synapsinGP_5thA',
 'psd_8thA',
 'dapi_1stA',
 'GABABR1_3rdA',
 'NMDAR1_6thA',
 'NOS_9thA',
 'gephyrin_1stA']

## Save as HDF5

In [60]:
h5fOUT = h5py.File(output, 'a')
#h5fOUT.create_dataset(TOKEN1, data=cols)
#h5fOUT.create_dataset(TOKEN1, data=np.transpose(synCube))
h5fOUT.create_dataset('Locations/new', data=np.transpose(L))
h5fOUT.create_dataset('Channels/new', data=columnNames)
h5fOUT.close()

RuntimeError: Unable to create link (Message type not found)